In [1]:
to_do = {'Поменять метрику с ф1 на рок-аук': True,
        'Добавить catboost': False,
        'установить Pandas-profiling': True,
        'Разобраться в Pandas-profiling': False}

# Описание, заметки, размышления

Для прогнозирования используются данные о покупках юзеров за период 2021 года. Всего доступно больше 2 млн. строк.

Флаг оттока проставляется, если после покупки юзера прошло больше 45 дней.

Поведение юзера описывается историей его покупок, суммой чека и размером скидки к чеку. В компании действует программа лояльности, участвуя в которой юзеры получают скидку на любую покупку и могут получать дополнительные скидки по разным промо-акциями.

Описание данных:
data.csv
* `'clnt_ID'` - уникальный айди юзера, str
* `'timestamp'` - дата и время совершения покупки, datetime
* `'gest_Sum'` - сумма покупки, float
* `'gest_Discount'` - сумма скидки, float

target.csv
* `'clnt_ID'` - уникальный айди юзера, str
* `'target'` - флаг оттока, int: 1 если юзер ушел в отток | 0 если НЕ отток

## Что здесь можно сделать? - Заметки 
**Нужно помнить, что по сути это тайм серия.**  -- **Внимательно!**  
Т.е. я предполагаю, что клиент покупает, покупает, покупает, потом фигак и перестал покупать. Один клиент - один вектор.
А тут куча вектором принадлежат одному клиенту. 

Т.е. я бы даже сказал, что в том виде в котором оно есть, модель особо-то ничего не найдет. Как она сможет по дате, сумме покупки и скидке предсказать уйдет человек или нет, не зная его истории?
---ОТВЕТ---
Да, это последовательность событий, но прям как таймсерию использовать не можем, потому что нерегулярные сигналы и из-за этого много сложностей.
Но! Можно использовать эти же строки, но к ним добавлять "память" о клиенте, т.е. его историю к конкретной сделке.

**Что может говорить, что клиент собрался уйти и не вернуться?**  
Т.е. гипотетически что может говорить о том, что клиент собирается уйти и не вернуться?
1. Увеличивается интервал между покупками от покупки к покупке. Типа 1 день, потом 2, потом 5 и т.д.
2. Как-то изменяется объем чека (уменьшается, увеличивается?) 
3. Как-то меняется ассортимент (типа вот он покупал одно и тоже, а тут вдруг перестал покупать одно и тоже (испортился товар на его вкус) и он либо сразу ушел, либо попробовал другое и остался, либо попробовал другое - ему не понравилось - ушел

**Как должен работать предикт? Какие данные должны поступать на вход?**  -- **Есть пол-ответа**  
~~Это видимо тоже относится к тому, что это своего рода таймсерия~~
Вся инфа находится в БД, которая обновляется ночью. И модель будет работать ночью и может использовать всю бд. Как именно пока я не понимаю.


**Если сейчас дать модели на вход просто дату, сумму чека и скидку...**  
То она будет пытаться предсказать отток исключительно по дате, сумме чеку и скидке - врядли у нее что-то получится взразумительное.


**Сумма в месяц стабильна для юзера, а отклонение может указывать на отток**  
По результатам уже проведенного анализа есть такой вывод:
сумма, которую готов тратить юзер в месяц,  достаточна стабильна и слабо меняется со временем. Потратив в этом месяце больше обычного, юзер скорее всего в следующем не будет покупать;

По каждому клиенту взять средний чек, каким-то образом определить порог отклонения от этого среднего чека и посмотреть насколько факт этого отклонения коррелирует с оттоком. Может это будет хорошая фича. Но проблема в том, что для предсказания не достаточно 

**Самый простой и очевидный способ подготовки данных**  
Это взять по каждому клиенту кол-во покупок, min,max,mean,median,stdev по gest_Sum, gest_Discount, и перерывом между покупками

**Вариант еще лучше!**  
Используем каждую строку как есть, не переводим ее в строку по уникальному клиенту, а точно также одна строка одна сделка.
К каждой такой строке мы накопительно добавляем кумулятивную инфу о клиенте, т.е.:
* Сколько дней на момент сделки прошло с момента первой сделки
* Какой на текущий момент средний чек
* Какая разница между среднем чеком и чеком сделки
* Сколько сделок произошло на момент текущей сделки
* Сколько сделок в месяц в среднем



## Вопросы:
**>>>Что значат 0 в gest_sum?<<<**

**Как работают скидки? Это накопительные баллы или просто скидки? Почему основные скидки не больше 100 руб, но бывают аж в 3000 руб.?**

## Описание общего подхода - not actual

Я вот вижу 3 общих подхода:
1. Рассматривать покупки клиентов как таймсерию, и исходя из прошлого поведения прогнозировать будущее. 
    * Мы таймсерии не проходили, плюс задача оттока традиционно не решается предсказанием таймсерии, поэтому от этого варианта скорее откажемся. Даже несмотря на то, что можно как-то обучить модель на скользящем окне (я точно не знаю как).
2. Преобразовать данные так, чтобы получилось по одной строке на каждого клиента, в которой будет какая-то агрегированная информация о покупках. В надежде, что модель найдет какие-то закономерности именно в общей репрезентации истории. 
3. Использовать данные в том виде, в котором они уже есть. Т.е. каждая строка это одна сделка. Но добавить к ним накопительную информацию об истории клиента. (Спасибо нашему ментору, который подсказал этот подход)
    * Т.е. к каждой сделке добавить инфу о кол-ве уже совершенных покупок, кол-во прошедших дней с предыдущей покупки, и другое.
    * Такой подход позволит модели сравнивать историю с текущей сделкой, что повышает ценность информации по сравнению с предыдущим подходом.
    
    
Наверное, главный мой вопрос для всех: какие есть еще подходы к решению задачи?

И перейдя на уровень ниже, предполагаю, что эффективность любого подхода зависит от того, какие фичи мы создадим/добавим.
А какие фичи нужно добавить зависит от того, как мы себе представляем механику и причины оттока клиентов.

Здесь можно как-то самостоятельно (коллективно) очертить эту механику, составить список фичей и уже отдать на откуп модели, чтобы она там сама определила какие фичи важны - какие нет.
Либо воспользоваться уже какой-то готовой теорией, например тот же RFM-анализ, о котором говорил Родион.
Не ясно какой подход эффективнее, т.е. стоит ли погружаться в теорию и если стоит то ради чего (только ради списка фичей, например)?
Это вот второй вопрос, который хотел бы обсудить.


И третий вопрос: про то, в каком виде будут приходить данные на предикт.
Это может повлиять на то, как нужно будет обучать модель.
Например, если предикт будет происходить исключительно по данным за последний месяц и только, то какой-то условный скрипт предобработки, который будет, например, считать средний чек может сильно ошибиться, т.к. ему будет не хватать данных. А модель мы обучали на целом годе и там статистики будут поточнее. И так возникает расхождение.
Вот такого рода ограничения. Я, к сожалению, другие примеры придумать не могу, но это все может сильно повлиять на архитектуру обработки данных и как следствие на то, что стоит ожидать от модели.

**not actual - end**

Структура:  
0. Описание, заметки, размышления
1. Импорты и настройки
2. Предобработка
3. EDA того, что есть
4. Feature engineering
5. New features EDA
6. Baseline and model selection
7. Model tuning

# Импорты и настройки

In [2]:
import numpy as np
import pandas as pd

import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns

from pandas_profiling import ProfileReport

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.metrics import f1_score


In [3]:
# pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:.3f}'.format

# Предобработка

In [4]:
df = pd.read_csv('data/data.csv')
df_target = pd.read_csv('data/target.csv')

In [5]:
display(df.head())
df_target.head()

,clnt_ID,timestamp,gest_Sum,gest_Discount
0,193B4268-0B4A-475E-B1D0-FF5515E29D29,2021-01-02 09:09:17.060,900.000,300.000
1,8DA65A37-C1D0-41D4-98E1-AB6C5BF1367F,2021-01-02 09:12:24.850,165.000,55.000
2,26ACF3C8-25C8-4345-ABC2-33DA15EA6454,2021-01-02 09:38:21.643,800.500,25.500
3,0F77DDB3-A9A7-44BE-AAAB-9DF59B66A695,2021-01-02 09:45:17.793,580.000,0.000
4,F16BCF77-FA5A-4093-B7E3-FA86E2B1EA31,2021-01-02 09:59:50.453,148.900,3.100


,clnt_ID,target
0,000070A8-DB9E-4AB7-8C4D-6169D4AEBB2A,1
1,00007EB0-6331-438E-A917-E9840C260876,0
2,0000993D-A30E-4233-AB3F-D368D9A0D2C4,1
3,0000A724-7BC5-408F-9F16-6CC3AB16322F,0
4,0000B90C-56DE-43C2-A213-624AFBE36DB2,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2371635 entries, 0 to 2371634
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   clnt_ID        object 
 1   timestamp      object 
 2   gest_Sum       float64
 3   gest_Discount  float64
dtypes: float64(2), object(2)
memory usage: 72.4+ MB


In [7]:
df_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255109 entries, 0 to 255108
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   clnt_ID  255109 non-null  object
 1   target   255109 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.9+ MB


* timestamp - object, а не datetime

Изменим тип данных в timestamp на datetime

In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

Добавим столбец с только датой.

In [9]:
df['date'] = df['timestamp'].dt.date

Проверим на дубликаты и пропуски.

In [10]:
df.isna().sum()

clnt_ID          0
timestamp        0
gest_Sum         0
gest_Discount    0
date             0
dtype: int64

Пропусков нет.

In [11]:
df.duplicated().sum()

129

Есть дубликаты, посмотрим на них.

In [12]:
df.loc[df.duplicated(keep=False)]

,clnt_ID,timestamp,gest_Sum,gest_Discount,date
639,1FA367BD-8E07-437E-AB6A-009D53533128,2021-01-02 12:00:00,982.200,0.000,2021-01-02
663,1FA367BD-8E07-437E-AB6A-009D53533128,2021-01-02 12:00:00,982.200,0.000,2021-01-02
85786,3319362D-F6AD-498B-9B53-F59D0418915E,2021-01-14 14:00:00,1189.000,0.000,2021-01-14
85787,3319362D-F6AD-498B-9B53-F59D0418915E,2021-01-14 14:00:00,1189.000,0.000,2021-01-14
133279,A145C6F8-EDCB-47B1-BF5D-B328F6AF97CA,2021-01-20 16:00:00,0.000,0.000,2021-01-20
...,...,...,...,...,...
2165277,E99EFF35-01BE-49A9-8F61-641F752051E5,2021-11-08 15:00:00,0.000,0.000,2021-11-08
2174627,1A4C0239-A7AB-4188-84DB-8F52FBF6E13C,2021-11-10 11:00:00,282.240,37.760,2021-11-10
2174629,1A4C0239-A7AB-4188-84DB-8F52FBF6E13C,2021-11-10 11:00:00,282.240,37.760,2021-11-10
2198034,C9D86AF9-373C-4531-92A4-8FC752DD5D75,2021-11-14 12:00:00,0.000,0.000,2021-11-14


Удалим дубликаты.

In [13]:
df = df.drop_duplicates()

In [14]:
df.duplicated().sum()

0

Дубликаты удалили.

# EDA того, что есть

Анализируем.
* Дата сет как есть
* Дата сет агрегированный по клиентам

## Анализируем дата сет как есть

In [24]:
# Обеъединим фичи с таргетами
df_eda = df.merge(df_target,on='clnt_ID')
# df_eda = df_eda.drop(columns=['timestamp'])

In [25]:
profile = ProfileReport(df_eda,minimal=True)

In [26]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

## Анализируем агрегированный по юзерам датасет

In [35]:
df_eda_agg_by_user = df.pivot_table(index='clnt_ID',aggfunc=['count','min','max','median','mean','std'])

In [36]:
df_eda_agg_by_user.columns = ['_'.join(column) for column in df_eda_agg_by_user.columns]
df_eda_agg_by_user.columns

Index(['count_date', 'count_gest_Discount', 'count_gest_Sum',
       'count_timestamp', 'min_date', 'min_gest_Discount', 'min_gest_Sum',
       'min_timestamp', 'max_date', 'max_gest_Discount', 'max_gest_Sum',
       'max_timestamp', 'median_gest_Discount', 'median_gest_Sum',
       'mean_gest_Discount', 'mean_gest_Sum', 'std_gest_Discount',
       'std_gest_Sum'],
      dtype='object')

In [39]:
df_eda_agg_by_user.head().reset_index()

,clnt_ID,count_date,count_gest_Discount,count_gest_Sum,count_timestamp,min_date,min_gest_Discount,min_gest_Sum,min_timestamp,max_date,max_gest_Discount,max_gest_Sum,max_timestamp,median_gest_Discount,median_gest_Sum,mean_gest_Discount,mean_gest_Sum,std_gest_Discount,std_gest_Sum
0,000070A8-DB9E-4AB7-8C4D-6169D4AEBB2A,3,3,3,3,2021-01-24,10.000,490.000,2021-01-24 13:45:46.533,2021-02-21,44.800,566.200,2021-02-21 13:47:54.810,10.600,519.400,21.800,525.200,19.921,38.430
1,00007EB0-6331-438E-A917-E9840C260876,1,1,1,1,2021-01-12,6.000,214.000,2021-01-12 09:39:50.887,2021-01-12,6.000,214.000,2021-01-12 09:39:50.887,6.000,214.000,6.000,214.000,NaN,NaN
2,0000993D-A30E-4233-AB3F-D368D9A0D2C4,1,1,1,1,2021-07-25,0.000,470.000,2021-07-25 15:03:30.863,2021-07-25,0.000,470.000,2021-07-25 15:03:30.863,0.000,470.000,0.000,470.000,NaN,NaN
3,0000A724-7BC5-408F-9F16-6CC3AB16322F,64,64,64,64,2021-01-19,0.000,0.000,2021-01-19 12:24:35.337,2021-11-26,280.800,562.600,2021-11-26 17:14:12.610,8.550,260.550,31.162,274.228,62.320,118.275
4,0000B90C-56DE-43C2-A213-624AFBE36DB2,1,1,1,1,2021-10-27,0.000,549.000,2021-10-27 13:42:00.000,2021-10-27,0.000,549.000,2021-10-27 13:42:00.000,0.000,549.000,0.000,549.000,NaN,NaN


In [40]:
profile = ProfileReport(df_eda_agg_by_used.reset_index(),minimal=True)
profile.to_widgets()

TypeError: Setting a MultiIndex dtype to anything other than object is not supported

In [42]:
new = df_eda_agg_by_user.merge(df_target,on='clnt_ID')


,clnt_ID,count_date,count_gest_Discount,count_gest_Sum,count_timestamp,min_date,min_gest_Discount,min_gest_Sum,min_timestamp,max_date,max_gest_Discount,max_gest_Sum,max_timestamp,median_gest_Discount,median_gest_Sum,mean_gest_Discount,mean_gest_Sum,std_gest_Discount,std_gest_Sum,target
0,000070A8-DB9E-4AB7-8C4D-6169D4AEBB2A,3,3,3,3,2021-01-24,10.000,490.000,2021-01-24 13:45:46.533,2021-02-21,44.800,566.200,2021-02-21 13:47:54.810,10.600,519.400,21.800,525.200,19.921,38.430,1
1,00007EB0-6331-438E-A917-E9840C260876,1,1,1,1,2021-01-12,6.000,214.000,2021-01-12 09:39:50.887,2021-01-12,6.000,214.000,2021-01-12 09:39:50.887,6.000,214.000,6.000,214.000,NaN,NaN,0
2,0000993D-A30E-4233-AB3F-D368D9A0D2C4,1,1,1,1,2021-07-25,0.000,470.000,2021-07-25 15:03:30.863,2021-07-25,0.000,470.000,2021-07-25 15:03:30.863,0.000,470.000,0.000,470.000,NaN,NaN,1
3,0000A724-7BC5-408F-9F16-6CC3AB16322F,64,64,64,64,2021-01-19,0.000,0.000,2021-01-19 12:24:35.337,2021-11-26,280.800,562.600,2021-11-26 17:14:12.610,8.550,260.550,31.162,274.228,62.320,118.275,0
4,0000B90C-56DE-43C2-A213-624AFBE36DB2,1,1,1,1,2021-10-27,0.000,549.000,2021-10-27 13:42:00.000,2021-10-27,0.000,549.000,2021-10-27 13:42:00.000,0.000,549.000,0.000,549.000,NaN,NaN,0


In [45]:
new.loc[(new['target']==0)&(new['count_date']==1)]

,clnt_ID,count_date,count_gest_Discount,count_gest_Sum,count_timestamp,min_date,min_gest_Discount,min_gest_Sum,min_timestamp,max_date,max_gest_Discount,max_gest_Sum,max_timestamp,median_gest_Discount,median_gest_Sum,mean_gest_Discount,mean_gest_Sum,std_gest_Discount,std_gest_Sum,target
1,00007EB0-6331-438E-A917-E9840C260876,1,1,1,1,2021-01-12,6.000,214.000,2021-01-12 09:39:50.887,2021-01-12,6.000,214.000,2021-01-12 09:39:50.887,6.000,214.000,6.000,214.000,NaN,NaN,0
4,0000B90C-56DE-43C2-A213-624AFBE36DB2,1,1,1,1,2021-10-27,0.000,549.000,2021-10-27 13:42:00.000,2021-10-27,0.000,549.000,2021-10-27 13:42:00.000,0.000,549.000,0.000,549.000,NaN,NaN,0
12,0001CEBE-0496-4642-A3A7-D544B61DCB02,1,1,1,1,2021-08-08,72.000,138.000,2021-08-08 12:57:21.967,2021-08-08,72.000,138.000,2021-08-08 12:57:21.967,72.000,138.000,72.000,138.000,NaN,NaN,0
17,0003266D-BB90-45D7-B551-FB3B3EB331A5,1,1,1,1,2021-10-03,99.000,891.000,2021-10-03 14:00:00.000,2021-10-03,99.000,891.000,2021-10-03 14:00:00.000,99.000,891.000,99.000,891.000,NaN,NaN,0
25,00046D12-C3BA-464E-A87D-92CB82DCD1AA,1,1,1,1,2021-04-16,0.000,1090.000,2021-04-16 20:34:13.903,2021-04-16,0.000,1090.000,2021-04-16 20:34:13.903,0.000,1090.000,0.000,1090.000,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255077,FFF79C51-497D-41A0-A9E8-8E87600301E9,1,1,1,1,2021-04-25,0.000,1084.000,2021-04-25 18:07:00.000,2021-04-25,0.000,1084.000,2021-04-25 18:07:00.000,0.000,1084.000,0.000,1084.000,NaN,NaN,0
255078,FFF7D539-3F90-4C35-BBA7-5C8A20EF8A22,1,1,1,1,2021-07-11,0.000,1000.000,2021-07-11 11:35:00.467,2021-07-11,0.000,1000.000,2021-07-11 11:35:00.467,0.000,1000.000,0.000,1000.000,NaN,NaN,0
255079,FFF92989-B4BC-48AF-92DE-603F1547631F,1,1,1,1,2021-10-07,12.000,55.000,2021-10-07 15:25:35.370,2021-10-07,12.000,55.000,2021-10-07 15:25:35.370,12.000,55.000,12.000,55.000,NaN,NaN,0
255082,FFF9BC13-DC72-4402-9E5F-606DED21972B,1,1,1,1,2021-11-23,0.000,426.000,2021-11-23 19:11:33.063,2021-11-23,0.000,426.000,2021-11-23 19:11:33.063,0.000,426.000,0.000,426.000,NaN,NaN,0


## OLD_EDA

Объединим датасеты для анализа.

In [ ]:

df_eda = df.merge(df_target,on='clnt_ID')
display(df_eda.head(),df_eda.info(),df_eda.isna().sum())

Посмотрим кол-во уникальный айдишников.

In [ ]:
display(df['clnt_ID'].nunique())
df_target['clnt_ID'].nunique()

255109 клиентов. При этом в датасете с таргетом тоже 255109 записей. Значит все верно.

Посмотрим сколько из них с флагом.

In [ ]:
df_target['target'].value_counts()

Примерно одинаково. Дисбаланса классов нет.

Проверим, что все правильно смержилось.

In [ ]:
(df_eda.loc[df_eda['target']==0,'clnt_ID'].nunique(),df_eda.loc[df_eda['target']==1,'clnt_ID'].nunique())

Да, кол-во уникальных айдишников в каждом классе осталось прежним.

Посмотрим сколько заказов (строк) приходится на каждый класс.

In [ ]:
display(df_eda['target'].value_counts())
sns.countplot(x='target',data=df_eda)
plt.show()

Тут мы видим сразу, что те, кто уходят, значительно меньше раз совершали покупки.

### Задачи EDA

Сначала нужно исследовать данные том виде, в котором пришли.
Т.е. это даты, сумма чека с учетом скидки, размер скидки

Потом можно посравнивать агрегированную информацию каждого клиенту по таргету.

### Сделаем функцию дескрайб мою

In [ ]:
def describe(col):
    """Функция выводит подробное числовое описание данных, 
    возвращает Figure,Axis объекты matplotlib.pyplot для гистограммы и ящика с усами.
    """

#     Подсчет количества значений
    count_ = col.count()
    
#     Меры центральной тенденции:
#     Среднее
    mean = col.mean()
#     Медиана
    median = col.median()
#     Мода
    mode = col.mode()

#     Меры разброса
#     Дисперсия
    var = col.var()
#     Стандартное отклонение
    stddev = col.std()
#     Межквартильный размах
    quartile_1 = col.quantile(0.25)
    quartile_3 = col.quantile(0.75)
    iqr = quartile_3 - quartile_1
#     Мин, макс, размах
    min_ = col.min()
    max_ = col.max()
    range_ = max_ - min_

#     Левая граница ящика с усами
    outlier_border_left = quartile_1 - 1.5 * iqr
#     Правая граница ящика с усами
    outlier_border_right = quartile_3 + 1.5 * iqr
    
#    Количество выбросов
    outliers_mask = (col <= outlier_border_left) | (col >= outlier_border_right)
    outliers_count = col.loc[outliers_mask].count()
    outliers_proportion = outliers_count / count_
    
#     Создание и вывод таблицы с числовым описанием данных
    describe_ = {"count": count_,
                 "mean": mean,
                 "mode": mode,
                 "var": var,
                 "stddev": stddev,
                 "min": min_,
                 "25%": quartile_1,
                 "50%": median,
                 "75%": quartile_3,
                 "max": max_,
                 "total_range": range_,
                 "interquartile_range": iqr,
                 "outlier_border_left": outlier_border_left,
                 "outlier_border_right": outlier_border_right,
                 "outliers_count": outliers_count,
                 "outliers_proportion": outliers_proportion}
    describe_ = pd.DataFrame(describe_).transpose()
    describe_.columns = [col.name]
    return describe_

# describe(t_0)

### Функция построения двух гистограмм на одной фигуре

In [ ]:
def multiv_target_hist_normalized(data_0,data_1,bins=1000,xlabel=None,figsize=(10,5)):
    """
    Построение двух полупрозрачных нормализованных гистограмм (с функцией плотности) на одной фигуре.
    Каждая гистограмма это подгруппа признака по таргету.
    
    """
    
#     canceled = data_train.loc[df['is_canceled'] == 1, column].sort_values()
#     not_canceled = data_train.loc[df['is_canceled'] == 0, column].sort_values()
    
    fig,ax = plt.subplots(figsize=figsize)
    
#     if bins == False:
#         bins = df[column].nunique()
        
    ax.hist(data_0,bins=bins,alpha=0.5,label=['target=0'])
    ax.hist(data_1,bins=bins,alpha=0.5,label=['target=1'])
    ax.set_ylabel('Частота')
    if xlabel == None:
        ax.set_xlabel("Сравнение статистики по категориям")
    else:
        ax.set_xlabel(xlabel)
    ax.legend()
    return fig,ax

# multiv_target_hist_normalized(t_0,t_1,bins=[1,2,3,10,100,200,500,1000,10000])
# plt.xlim(0,200)

### Исследуем данные как есть

**timestamp и date**

In [ ]:
df['timestamp'].describe(datetime_is_numeric=True)

In [ ]:
df.loc[df['timestamp'].duplicated(keep=False)]

In [ ]:
df['timestamp'].hist(bins=365)

In [ ]:
df['date'].describe()

In [ ]:
df['date'].hist(bins=365)

* Да ничего неожиданного. Что я тут вообще ожидал найти?

**gest_Sum**

In [ ]:
# fig,ax = simple_uni_EDA(df,"gest_Sum")
# ax.set_xlabel("Сумма чека за вычетом скидки")
# ax.set_title("Гистограмма и ящик с усами суммы чека за вычетом скидки")
# ax.set_xlim(0,5000)
# plt.show()
# **ПРОБЛЕМА С ФУНКЦИЕЙ**
# Мне тут не нравится, что нельзя изменять кол-во бинов.
# **ПРОБЛЕМА С ФУНКЦИЕЙ**

df['gest_Sum'].describe()

Есть 0. Что это значит?

Посмотрим на эти строки, где `'gest_Sum' == 0`

In [ ]:
df.loc[df['gest_Sum']==0].describe()

* Возможно это отмена заказа?

In [ ]:
df['gest_Sum'].hist(bins=10000)
plt.xlim(0,200)
plt.show()

Есть какой-то странный пик в районе от 75 до 100 руб. Это покупка кофе?

In [ ]:
df.loc[(df['gest_Sum'] > 75)&(df['gest_Sum'] < 100)].describe()

Сложно сказать что-то конкретное.

In [ ]:
df['gest_Sum'].hist(bins=10000)
plt.xlim(0,1500)
plt.show()

Что тут еще добавить? Я не знаю.

**gest_Discount**

In [ ]:
df['gest_Discount'].describe()

In [ ]:
df['gest_Discount'].hist(bins=1000)
plt.xlim(0,100)

Ну как бы и чо? Я не понимаю что я тут должен найти, кроме того что это не нормальное распределение.

### Исследуем агрегированные данные

У нас есть кол-во сделок; мин, макс, среднее, медиана, мода, стандартное отклонение `'gest_Sum'`, `'gest_Discount'`. И нам надо их сравнить по таргету, чтобы выяснить как эти статистики отличаются по категориям.

Напишем функцию, которая будет подготавливать такие датасеты со статистиками по категориям.

In [ ]:
def cat_split(value,aggfunc):
    pivot_table = df_eda.pivot_table(index='clnt_ID', values=value, aggfunc=aggfunc,columns='target')
    target_0 = pivot_table[0].dropna()
    target_1 = pivot_table[1].dropna()
    return target_0, target_1

# cat_split('date','count')

In [ ]:
t_0, t_1 = cat_split('date','count')
describe(t_0).join(describe(t_1))

In [ ]:
describe(t_0).join(describe(t_1))

In [ ]:
sns.histplot(t_0)
plt.xlim(0,100)

In [ ]:
# В итоге мы пока откажемся от подхода с таймсериес, и возьмем просто описание истории покупок клиентов.
# Т.е. берем фичи сумма чека, скидка, дней с последней покупки для каждого клиента и сравнимаем эти статистики по группам 0 и 1 (таргету)
# df_eda.pivot_table(index='clnt_ID', values=['gest_Sum','gest_Discount'],aggfunc=['count','min','max','median','mean','std'],columns='target')

Напишем функцию для упрощения анализа статистик по категориям.

In [ ]:
def simple_uni_EDA(df,column):
    """Функция выводит подробное числовое описание данных, 
    возвращает Figure,Axis объекты matplotlib.pyplot для гистограммы и ящика с усами.
    """

#     Подсчет количества значений
    count_ = df[column].count()
    
#     Меры центральной тенденции:
#     Среднее
    mean = df[column].mean()
#     Медиана
    median = df[column].median()
#     Мода
    mode = df[column].mode()

#     Меры разброса
#     Дисперсия
    var = df[column].var()
#     Стандартное отклонение
    stddev = df[column].std()
#     Межквартильный размах
    quartile_1 = df[column].quantile(0.25)
    quartile_3 = df[column].quantile(0.75)
    iqr = quartile_3 - quartile_1
#     Мин, макс, размах
    min_ = df[column].min()
    max_ = df[column].max()
    range_ = max_ - min_

#     Левая граница ящика с усами
    outlier_border_left = quartile_1 - 1.5 * iqr
#     Правая граница ящика с усами
    outlier_border_right = quartile_3 + 1.5 * iqr
    
#    Количество выбросов
    outliers_mask = (df[column] <= outlier_border_left) | (df[column] >= outlier_border_right)
    outliers_count = df.loc[outliers_mask,column].count()
    outliers_proportion = outliers_count / count_
    
#     Создание и вывод таблицы с числовым описанием данных
    describe_ = {"count": count_,
                 "mean": mean,
                 "mode": mode,
                 "var": var,
                 "stddev": stddev,
                 "min": min_,
                 "25%": quartile_1,
                 "50%": median,
                 "75%": quartile_3,
                 "max": max_,
                 "total_range": range_,
                 "interquartile_range": iqr,
                 "outlier_border_left": outlier_border_left,
                 "outlier_border_right": outlier_border_right,
                 "outliers_count": outliers_count,
                 "outliers_proportion": outliers_proportion}
    describe_ = pd.DataFrame(describe_).transpose()
    describe_.columns = [column]
    with pd.option_context("display.float_format", '{:.3f}'.format):
                           display(describe_)
    

#     Построение гистограммы, ящика с усами
    fig,ax = plt.subplots(figsize=(16,8))
    ax.hist(df[column],bins=30)
    ax.set_ylabel("Частота")
    ax2=ax.twinx()
    ax2.boxplot(df[column],vert=False)
    
    return fig,ax

def qqplot(df,column):
    """Функция возвращает Figure,Axis объекты matplotlib.pyplot для normal qqplot."""
    fig,ax = plt.subplots(figsize=(8,8))
    sm.qqplot(df[column], line='45',fit=True,ax=ax)
    return fig,ax

In [ ]:
fig,ax = simple_uni_EDA(df,"gest_Sum")
ax.set_xlabel("Кол-во звонков")
ax.set_title("Гистограмма и ящик с усами кол-ва звонков")
plt.show()
# fig,ax = qqplot(df,"gest_Sum")
# plt.show()

In [ ]:
def eda(value,aggfunc,bins):
    eda = df_eda.pivot_table(index='clnt_ID',values=value,aggfunc=aggfunc,columns='target')
    t_0 = eda.iloc[:,0].dropna()
    t_1 = eda.iloc[:,1].dropna()
    display(t_0.value_counts())
    display(t_1.value_counts())
    return plt.hist([t_0,t_1],bins=bins,label=['target=0','target=1'])
    
    
eda('gest_Sum','count',200)
plt.xlim(0,200)
plt.legend()
plt.show()

# Feature engineering

Для анализа нужно создать фичи

**=====================================================**  
**=====================================================**  
**=====================================================**  
**СТАВИМ НА ПАУЗУ, СНАЧАЛА СДЕЛАТЬ EDA БЕЗ НОВЫХ ФИЧЕЙ, И БЕЙСЛАЙН, ЗАТЕМ УЖЕ НОВЫЕ ФИЧИ И НОВЫЕ МОДЕЛИ**  
**=====================================================**  
**=====================================================**  
**=====================================================**  

Общая стоимость заказа без скидки = `gest_Sum + gest_Discount`

In [ ]:
# df['gest_total'] = df['gest_Sum'] + df['gest_Discount']

**Как создавать сложные фичи?**

* У нас есть целиковый датасет
* Нужно группировать его по клиентам
* Нужно к этим группам применять разные методы, чтобы уже эти фичи создать

`df.groupby('cnlt_ID').transform(func)`

**Какие создавать фичи?**
* Сколько дней на момент текущей сделки прошло с момента первой сделки
* Какой на текущий момент средний чек
* Какая разница между среднем чеком и чеком сделки
* Сколько сделок произошло на момент текущей сделки
* Сколько сделок в месяц в среднем

Сколько дней прошло с момента первой сделки

1. Берем min(date)
2. Вычитаем из текущей data min(date) (`df['date'] - df['first_buy_date']`)

In [ ]:
# def first_buy_date(col):
# #     print(col.min())
#     return col - col.min()
# df['first_buy_date_delta'] = df.groupby('clnt_ID')['date'].transform(first_buy_date)
# df['first_buy_timestamp_delta'] = df.groupby('clnt_ID')['timestamp'].transform(first_buy_date)

In [ ]:
# df['first_buy_date_delta'].dt.days.astype('int').hist(bins=100)

In [ ]:
# df['first_buy_timestamp_delta'].dt.total_seconds().astype('int').hist(bins=100)

**Какой на текущий момент средний чек**

Берем кумулятивную сумму чека и делим на 

А лучше используем `series.expanding().mean()`
`df.groupby('clnt_ID').transform()`

При этом нам нужен среднее кол-во денег, которое отдал клиент, и средняя стоимость без учета скидки.

In [ ]:
# def expand_mean(col):
#     return col.expanding().mean()
# df['clnt_gest_Sum_avg'] = df.groupby('clnt_ID')['gest_Sum'].transform(expand_mean)
# df['clnt_gest_total_avg'] = df.groupby('clnt_ID')['gest_total'].transform(expand_mean)

In [ ]:
# df.head()

In [ ]:
# df['clnt_gest_Sum_avg'].hist(bins=100)

**Какая разница между среднем чеком и чеком сделки**
Считаем для кол-ва денег и для стоимости без учета скидки (пока не ясно надо ли)

Берем текущий чек и из него вычитаем средний.
Тоже самое без учета скидки.

`df['gest_Sum'] - df['clnt_gest_Sum_avg']`
`df['gest_total'] - df['clnt_gest_total_avg']`

In [ ]:
# df['gest_Sum_delta_from_avg'] = df['gest_Sum'] - df['clnt_gest_Sum_avg']
# df['gest_total_delta_from_avg'] = df['gest_total'] - df['clnt_gest_total_avg']

In [ ]:
# df['gest_total_delta_from_avg'].hist(bins=100)

**Сколько сделок произошло на момент текущей сделки**
series.expanding().count()

In [ ]:
# def expand_count(col):
#     return col.expanding().count()

# df['clnt_buys_count'] = df.groupby('clnt_ID')['timestamp'].transform(expand_count)

In [ ]:
# df['clnt_buys_count'].describe()

In [ ]:
# df['clnt_buys_count'].hist(bins=100)

**=====================================================**  
**=====================================================**  
**=====================================================**  
**СТАВИМ НА ПАУЗУ, СНАЧАЛА СДЕЛАТЬ EDA БЕЗ НОВЫХ ФИЧЕЙ, И БЕЙСЛАЙН, ЗАТЕМ УЖЕ НОВЫЕ ФИЧИ И НОВЫЕ МОДЕЛИ**  
**=====================================================**  
**=====================================================**  
**=====================================================**  

# EDA включая новые фичи

# BASELINE и model selection

Для бейслайна используем данные в таком виде:
Группируем по клиенту,
берем общее число покупок
мин,макс,медиана,среднее,стд отклонение по gest_Sum, gest_Discount, кол-во дней между покупками.

Пробуем логистическую регрессию - для нее стандартизируем
Пробуем дерево решение - можно ничего не делать с данными
Пробуем случайный лет - тоже можно ничего не делать

Оцениваем ф1 по кроссвалидации на 5 фолдах (?)

In [ ]:
df_baseline = df.copy()

Создадим фичу - кол-во дней между покупками.
Потом ее переместим в нужное место.

In [ ]:
def expand_count(col):
    return col.expanding().count()

df_baseline['clnt_buys_count'] = df_baseline.groupby('clnt_ID')['timestamp'].transform(expand_count)

In [ ]:
df_baseline = df_baseline.drop(columns=['timestamp'])

In [ ]:
df_baseline.head()

Как использовать дату

In [ ]:
df_eda.pivot_table(index='clnt_ID', values=['gest_Sum','gest_Discount'],aggfunc=['count','min','max','median','mean','std'],columns='target')

In [ ]:
df_baseline.columns

In [ ]:
aggfunc_dict = {'date': ['count'],
                'gest_Sum': ['min','max','median','mean','std'], 
                'gest_Discount': ['min','max','median','mean','std'], 
                'clnt_buys_count': ['max','median','mean','std']} # здесь min не нужен, т.к. везде будут 1

In [ ]:
pivot_baseline = df_baseline.pivot_table(index='clnt_ID', values=['date','gest_Sum','gest_Discount','clnt_buys_count'],aggfunc=aggfunc_dict, fill_value=0)

In [ ]:
pivot_baseline.columns = ['_'.join(column) for column in pivot_baseline.columns]

In [ ]:
pivot_baseline.describe()

In [ ]:
pivot_baseline.isna().sum()

In [ ]:
# pivot_baseline_full = pivot_baseline.join(df_target)
pivot_baseline_full = pivot_baseline.merge(df_target,on='clnt_ID')

In [ ]:
pivot_baseline_full = pivot_baseline_full.drop(columns=['clnt_ID'])
# pivot_baseline_full.head()

In [ ]:
X = pivot_baseline_full.drop(columns=['target'])
y = pivot_baseline_full['target']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
log_reg = LogisticRegression()
log_reg_cv = cross_validate(log_reg, X, y, cv=5, scoring='roc_auc')

In [ ]:
tree = DecisionTreeClassifier()
tree_cv = cross_validate(tree,X,y,cv=5,scoring='roc_auc')

In [ ]:
forest = RandomForestClassifier()
forest_cv = cross_validate(forest,X,y,cv=5,scoring='roc_auc')

In [ ]:
tree_cv['test_score'].mean()

In [ ]:
log_reg_roc_auc = log_reg_cv['test_score'].mean()
tree_roc_auc = tree_cv['test_score'].mean()
forest_roc_auc = forest_cv['test_score'].mean()
print(f'log_reg_roc_auc = {log_reg_roc_auc} /n tree_roc_auc = {tree_roc_auc} /n forest_roc_auc={forest_roc_auc}')

### catboost -- ON HOLD

In [ ]:
# ON HOLD

# test_data = catboost_pool = Pool(train_data, 
#                                  train_labels)

# model = CatBoostClassifier(iterations=2,
#                            depth=2,
#                            learning_rate=1,
#                            loss_function='Logloss',
#                            verbose=True)
# # train the model
# model.fit(train_data, train_labels)
# # make the prediction using the resulting model
# preds_class = model.predict(test_data)
# preds_proba = model.predict_proba(test_data)
# print("class = ", preds_class)
# print("proba = ", preds_proba)

# Model tuning